In [12]:
import pyspark
from pyspark.streaming import StreamingContext
import pyspark_cassandra
# sc = pyspark.SparkConf()\
#     .setMaster("local[*]")\
#     .set("spark.driver.memory","8g")\
#     .set("spark.executor.memory","8g")\
#     .set("spark.debug.maxToStringFields", 10000) \
#     .setMaster("spark://spark-master:7077") \
#     .set("spark.cassandra.connection.host", "cas-1")
# sparkContext = pyspark.SparkContext(conf=sc)
# sc = CassandraSparkContext(conf=conf)

spark = pyspark.sql.SparkSession.builder\
    .appName("pysaprk_python")\
    .config('spark.cassandra.connection.host', 'localhost') \
    .config('spark.cassandra.connection.port', '9042') \
    .getOrCreate()

from pyspark import StorageLevel
from pyspark.sql.types import *
from itertools import chain
import redis, json, time
myRedis = redis.Redis(host='127.0.0.1', port=6379, db=0)

IP = "127.0.0.1"
Port = 5559
# cassandra 에서 받아오는 data schema
schema = StructType([
        StructField("created_at", StringType(), False),\
        StructField("id", LongType(), False),
        StructField("truncated", BooleanType(), False),
        StructField("retweeted", BooleanType(), False),
        StructField("reply_status", BooleanType(), False),
        StructField("is_quote_status", BooleanType(), False),
        StructField("media_status", BooleanType(), False),
        StructField("text", StringType(), False),
        StructField("user", StringType(), False),
        StructField("retweeted_status", StringType(), True),
        StructField("quoted_status", StringType(), True),
        StructField("entities", StringType(), False),
        StructField("extended_entities", StringType(), True),
        StructField("extended_tweet", StringType(), True),
        StructField("lang", StringType(), False),
        StructField("date", DateType(), False),
        StructField("timestamp_ms", TimestampType(), False)
    ])

myschema = [
    'entities.hashtag.text as hashtag'
]
option_schema = [
    'extended_entities.hashtag.text as extended_hashtag',
    'quoted_status.entities.hashtag.text as quoted_hashtag'
]
# 분석 대상에서 제외할 단어 명시
mystopwords = [
    'RT',
    'BTS',
    'bts',
    'Bts',
    '방탄소년단'
]
# get DStream RDD
def getStreaming(data, schema=None):
    data.pprint() # 실시간으로 들어오는 tweet 출력
    data.foreachRDD(process) # 각 rdd 별로 처리
    return True

# hashtag 전처리
def hashtag_processing(text):
    total = list(chain.from_iterable(text)) # 리스트 안에 리스트 하나의 리스트로 합치기
    result = []
    # 불용어 제거
    for i in total:
        if i not in mystopwords:
            result.append(i)

    print('응 들어옴')
    print(result)
    return result

# Convenience function for turning JSON strings into DataFrames.
def process(rdd):
    try:
        # json 형식으로 들어온 data 중 필요한 데이터만 추출
        data = json.loads(rdd)
        print(data)
        if data['truncated'] == 'true' and data['extended_entities']['hashtag']['text']:
            myschema.append(option_schema[0])
        if data['is_quoted_staus'] == 'true' and \
                data['quoted_status']['entities']['hashtag']['text']:
            myschema.append((option_schema[1]))
        print(myschema)

        rawTweet = spark.read.json(rdd)
        rawTweet.registerTempTable("tweets") #creates an in-memory table that is scoped to the cluster in which it was created.
        hashtag = rawTweet.selectExpr(myschema).rdd.flatMap(lambda x : x)
        print(hashtag.collect())
        # 현재 타임에 들어온 hashtag 전처리
        result = hashtag_processing(hashtag.collect())

        #word count 작업을 위해 결과 rdd로 만들어줌
        rdd = spark.sparkContext.parallelize(result)
        word_count(rdd)

    except:
        pass

# 추출된 단어 word count
def word_count(list):
    print('word count 들어옴')
    pairs = list.map(lambda word: (word, 1))
    # 상위 10개만 가져오기 + 등장빈도 2번 이상
    wordCounts = pairs.reduceByKey(lambda x, y: x + y).filter(lambda args : args[1] > 2)
    ranking = wordCounts.takeOrdered(10, lambda args:-args[1])
    print(ranking)
    # key : 현재 시간 , value : 순위 결과 json 으로 redis 저장
#     current_time = time.strftime("%d/%m/%Y")
#     rank_to_json = json.dumps(ranking)
#     myRedis.set(current_time, rank_to_json, ex=60*5)

ssc = StreamingContext(spark.SparkContext, 20)
#lines = ssc.socketTextStream(IP, Port)
#ssc.checkpoint("checkpoint")
lines = ssc.cassandraTable("test","testing")
getStreaming(lines)
ssc.start()
ssc.awaitTermination()
   


ModuleNotFoundError: No module named 'pyspark_cassandra'